In [ ]:
# Obtiene una máscara para cada municipio del país.

!pip install bottleneck

import pandas as pd
import numpy as np

import xarray as xr

from multiprocessing import Pool

In [ ]:
# Funciones para datos actuales.

# Obtiene el promedio temporal.
def proc_mean(year):
    # Abrimos el netCDF del año y variable correspondiente.
    with xr.open_dataset( path_d[0] + v + "mexico_" 
        + var + "." + str(year) + ".nc" ) as ds:
        # Establecemos np.nan como valor sin datos.
        ds = ds.where(ds > -9000)
        df_y = df[ df["Año"] == year ].copy()
        for i in df["CVE_INEGI"]:
            # Obtenemos el promedio dentro
            # de la máscara del municipio.
            df_y.loc[ df["CVE_INEGI"] == i,
                var ] = ( ds.where(mask
                .sel(CVE_INEGI = i) == 1 )
                .mean(dim = ["lat", "lon"])
                .to_dataframe().mean()[0] )
        return df_y

# Obtiene la suma temporal.
def proc_sum(year):
    # Abrimos el netCDF del año y variable correspondiente.
    with xr.open_dataset( path_d[0] + v + "mexico_" 
        + var + "." + str(year) + ".nc" ) as ds:
        df_y = df[ df["Año"] == year ].copy()
        for i in df["CVE_INEGI"]:
            # Obtenemos el promedio dentro
            # de la máscara del municipio.
            df_y.loc[ df["CVE_INEGI"] == i,
                var ] = ( ds.where(mask
                .sel(CVE_INEGI = i) == 1 )
                .mean(dim = ["lat", "lon"])
                .to_dataframe().sum()[0] )
        return df_y

In [ ]:
# Funciones para datos futuros.

# Obtiene el promedio temporal.
def proc_mean_f():
    # Abrimos el netCDF del año y variable correspondiente.
    with xr.open_dataset( path_d[k + 1] + v + "mexico_" 
        + var + "." + str(f) + "_daily.nc" ) as ds:
        # Establecemos np.nan como valor sin datos.
        ds = ds.where(ds > -9000)
        df_y = df.copy()
        for i in df["CVE_INEGI"]:
            # Obtenemos el promedio dentro
            #  de la máscara del municipio.
            df_y.loc[ df["CVE_INEGI"] == i,
                var ] = ( ds.where(mask
                .sel(CVE_INEGI = i) == 1 )
                .mean(dim = ["lat", "lon"])
                .to_dataframe().mean()[0] )
        return df_y

# Obtiene la suma temporal.
def proc_sum_f():
    # Abrimos el netCDF del año y variable correspondiente.
    with xr.open_dataset( path_d[k + 1] + v + "mexico_" 
        + var + "." + str(f) + "_daily.nc" ) as ds:
        df_y = df.copy()
        for i in df["CVE_INEGI"]:
            # Obtenemos el promedio dentro
            #  de la máscara del municipio.
            df_y.loc[ df["CVE_INEGI"] == i,
                var ] = ( ds.where(mask
                .sel(CVE_INEGI = i) == 1 )
                .mean(dim = ["lat", "lon"])
                .to_dataframe().sum()[0] )
        return df_y

# Obtiene la suma temporal y espacial.
def proc_soc_f():
    # Abrimos el netCDF del año y variable correspondiente.
    with xr.open_dataset( path_d[k + 1] + v + "mexico_" 
        + var + "." + str(f) + ".nc" ) as ds:
        df_y = df.copy()
        for i in df["CVE_INEGI"]:
            # Obtenemos el promedio dentro
            #  de la máscara del municipio.
            df_y.loc[ df["CVE_INEGI"] == i,
                var ] = ( ds.where(mask
                .sel(CVE_INEGI = i) == 1 )
                .sum("lat").to_dataframe()
                .sum()[0] )
        return df_y

In [ ]:
path_drive = "/content/drive/MyDrive/Colab/Subsidio_electricidad/" 

# Escenarios y años.
RCP = ["RCP4p5", "RCP8p5"]
fut = [2030, 2050, 2070]
years = list(range(2010, 2017))

# Directorios.
dirs = ["Actual"] + RCP
path_data = [path_drive + "data/" + x + "/" for x in dirs]
folder = "vars/"
path_d = [path_drive + folder + x + "/" for x in dirs]
v_0 = "/"

# Variables para generar la máscara.
vars = ["Tmax", "Tmin", "Tmean", "Pre",
        "HDD_mean", "CDD_mean", "HDD_p10",
        "CDD_p90", "Pre_Tmean", "Poblacion", "PIB"]

# Nombre de los archivos.
f_mask = "municipios_mask.nc"
csv = ".csv"

# Máscara para los municipios.
mask_0 = xr.open_dataset( path_drive + f_mask )

In [ ]:
# Escenario a procesar, 0: presente, 1: futuro.
e = 1

# Variable a procesar, con e = 0, 0 - 8; con e = 1, 0 - 10.
n = 10
var = vars[n]

mask = mask_0.copy()

if var in vars[4:8]:
    mask = mask.rename_vars(mask = "degree_day")
else:
    mask = mask.rename_vars(mask = var)

# Procesamos los años en paralelo.
pool = Pool()
# Datos actuales.
if e == 0:
    # Años a procesar.
    year = list(range(2010, 2017))

    # Abrimos los archivos a utilizar.
    df = pd.read_csv( path_data[0]
        + "data_0" + csv)
    v = v_0

    # Variables que se suman.
    if var in vars[3:9]:
        v = var + v
        df_r = pool.map(proc_sum, year)
    # Variables que se promedian.
    else:
        df_r = pool.map(proc_mean, year)

    # Concatenamos los resultados de cada año.
    for p in df_r:
        df.loc[df["Año"] == p["Año"].max() ] = p

    # Guardamos el resultado como un nuevo archivo.
    df.to_csv(path_data + "data_" + str(n + 1)
        + csv, index = False)
    
# Datos futuros.
else: 
    # Para el PIB y la población,
    # se agrega un año actual.
    if n in [9, 10]:
        fut = [2010] + fut
    
    # Iteramos en los RCPs.
    for k in range(len(path_d[1:])):
        
        # Iteramos en los años.
        for f in fut:
            b = 1

            # Abrimos los archivos a utilizar.
            df = pd.read_csv( path_data[k + 1] +
                "data_0." + str(f) + csv )
            v = v_0

            # Variables que se suman.
            if var in [vars[3], vars[8]]:
                df_r = proc_sum_f()
            elif var in vars[4:8]:
                # La variable está dentro
                # de una carpeta.
                v = var + v_0
                df_r = proc_sum_f()
            # Población  PIB
            elif var in vars[9:11]:
                # Por el momento, solo para RCP8.5
                if k != 0:
                    fut = [2010] + fut
                    df_r = proc_soc_f()
                else: b = 0
            # Variables que se promedian.
            else:
                df_r = proc_mean_f()

            if b:
                # Guardamos el resultado como un nuevo archivo.
                df_r.to_csv(path_data[k + 1] + "data_" + str(n + 1)
                    + "." + str(f) + ".csv", index = False)